# Efficient Yelp API Calls
- Daniel Barella
- 2/1/23

## For this assignment, you will be working with the Yelp API.

- As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

- Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

- Save your notebook, commit the change to your repository and submit the repository URL for this assignment.



In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
import json
with open('/Users/danielbarella/.secret/yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set our API call parameters 
LOCATION = 'Portland,OR'
TERM = 'Pizza'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Portland_pizza.json"
JSON_FILE

'Data/results_in_progress_Portland_pizza.json'

In [5]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Portland_pizza.json not found. Saving empty list to file.


In [6]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [7]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
## How many results total?
total_results = results['total']
total_results

1700

In [9]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [10]:
# Import additional packages for controlling our loop
#import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

85

In [11]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [12]:
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/85 [00:00<?, ?it/s]

In [13]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [14]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_Portland_pizza.json already exists. Deleting previous file...
[i] Data/results_in_progress_Portland_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


85

In [15]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/85 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [16]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,ywxapaOnDn3wyof-35BUIQ,life-of-pie-pizza-portland,Life of Pie Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/LhGaAT...,False,https://www.yelp.com/biz/life-of-pie-pizza-por...,846,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 45.549394, 'longitude': -122.666518}",[delivery],$$,"{'address1': '3632 N Williams Ave', 'address2'...",+15038200083,(503) 820-0083,1217.933062
1,-loAMvamhl-6kGw9jnetww,lovelys-fifty-fifty-portland,Lovely's Fifty Fifty,https://s3-media3.fl.yelpcdn.com/bphoto/K-GFEO...,False,https://www.yelp.com/biz/lovelys-fifty-fifty-p...,443,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 45.5528680273715, 'longitude': -1...",[delivery],$$,"{'address1': '4039 Mississippi Ave N', 'addres...",+15032814060,(503) 281-4060,2020.077922
2,Sw7ad82uHY6WGbu7TAFXBQ,wild-child-pizza-portland,Wild Child Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/h-XzPj...,False,https://www.yelp.com/biz/wild-child-pizza-port...,14,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 45.55896463063948, 'longitude': -...","[delivery, pickup]",NaN,"{'address1': '2032 NE Alberta St', 'address2':...",+15037197328,(503) 719-7328,1966.856217
3,Nr0cDHVHAgfGqrqw5x8ZaA,pizza-thief-portland,Pizza Thief,https://s3-media2.fl.yelpcdn.com/bphoto/DhxaXE...,False,https://www.yelp.com/biz/pizza-thief-portland?...,113,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,"{'latitude': 45.53659869202158, 'longitude': -...",[delivery],$$,"{'address1': '2610 NW Vaughn St', 'address2': ...",+15037197778,(503) 719-7778,4042.713209
4,o5RwFoFwK0HeqriEQi30Pg,pizzeria-otto-portland,Pizzeria Otto,https://s3-media2.fl.yelpcdn.com/bphoto/_Y8QFy...,False,https://www.yelp.com/biz/pizzeria-otto-portlan...,358,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 45.5462419841536, 'longitude': -1...",[delivery],$$,"{'address1': '6708 NE Sandy Blvd', 'address2':...",+19713738348,(971) 373-8348,4668.798142


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,YcNL6pQPz2P26WiT05a0ww,little-caesars-pizza-portland-19,Little Caesars Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/3ynVbH...,False,https://www.yelp.com/biz/little-caesars-pizza-...,28,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",2.0,"{'latitude': 45.50601, 'longitude': -122.495184}",[],$,"{'address1': '16353 SE Division St', 'address2...",+15037611141,(503) 761-1141,13054.311361
996,GLv7PEqqPrq1g5L1IezVPg,colwood-golf-center-portland-2,Colwood Golf Center,https://s3-media1.fl.yelpcdn.com/bphoto/EbeTTi...,False,https://www.yelp.com/biz/colwood-golf-center-p...,34,"[{'alias': 'golf', 'title': 'Golf'}, {'alias':...",3.5,"{'latitude': 45.56668978708045, 'longitude': -...",[],NaN,"{'address1': '7313 NE Columbia Blvd', 'address...",+15032545515,(503) 254-5515,5855.932201
997,1z-8TpwLRGZF6tDehvIWyQ,wilsonville-old-church-and-pub-wilsonville,Wilsonville Old Church & Pub,https://s3-media1.fl.yelpcdn.com/bphoto/kaVLjS...,False,https://www.yelp.com/biz/wilsonville-old-churc...,289,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",3.0,"{'latitude': 45.3011099, 'longitude': -122.773...",[],$$,"{'address1': '30340 SW Boones Ferry Rd.', 'add...",+15034272500,(503) 427-2500,28428.159208
998,ZF3psJYkJp_lCkKW2Ov7Rw,papa-johns-pizza-hillsboro,Papa Johns Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/SG0MLa...,False,https://www.yelp.com/biz/papa-johns-pizza-hill...,29,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 45.530086, 'longitude': -122.961358}","[delivery, pickup]",$,"{'address1': '2020 NE Cornell Rd', 'address2':...",+15036153131,(503) 615-3131,23984.139829
999,eojSyktwLt0tS9mLxTyiKA,lents-international-farmers-market-portland,Lents International Farmers Market,https://s3-media3.fl.yelpcdn.com/bphoto/IcqAXF...,False,https://www.yelp.com/biz/lents-international-f...,14,"[{'alias': 'farmersmarket', 'title': 'Farmers ...",4.5,"{'latitude': 45.481323, 'longitude': -122.568605}",[],$,"{'address1': 'SE 92nd & Reedway', 'address2': ...",+15032410032,(503) 241-0032,9539.754084
